**HOW TO CALL THE CLASS FROM YOUR NOTEBOOK**

In [6]:
from api.mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json

In [7]:
api = MapboxAPI()

In [8]:
response = api.get_data((41.97557881655979, -87.71361314512697), 15)
response

[[-87.764613, 42.097237],
 [-87.764624, 42.095568],
 [-87.763706, 42.095486],
 [-87.763657, 42.094535],
 [-87.762796, 42.094396],
 [-87.762685, 42.092507],
 [-87.761896, 42.092296],
 [-87.761722, 42.09147],
 [-87.761001, 42.091191],
 [-87.760775, 42.089417],
 [-87.758967, 42.087579],
 [-87.759263, 42.081228],
 [-87.759613, 42.08089],
 [-87.761613, 42.081441],
 [-87.762613, 42.079932],
 [-87.764469, 42.079579],
 [-87.762613, 42.079455],
 [-87.761613, 42.078659],
 [-87.759491, 42.078701],
 [-87.759199, 42.071579],
 [-87.758144, 42.069048],
 [-87.757467, 42.068579],
 [-87.757211, 42.066981],
 [-87.756541, 42.066506],
 [-87.757021, 42.064986],
 [-87.758613, 42.064968],
 [-87.760613, 42.062974],
 [-87.761613, 42.063522],
 [-87.763613, 42.06281],
 [-87.764263, 42.062929],
 [-87.764613, 42.063621],
 [-87.765613, 42.06293],
 [-87.768064, 42.06303],
 [-87.769272, 42.062579],
 [-87.761199, 42.061993],
 [-87.760671, 42.058637],
 [-87.761239, 42.058579],
 [-87.760562, 42.058579],
 [-87.759688, 42.